In [85]:
import numpy as np
from cvxopt import matrix, solvers
import time
def sdp(X,y,z,gamma):
    time1 = time.time()
    (m,n) = X.shape
    Y = np.concatenate([X, np.ones((m,1))], axis = 1)
    A11 = np.dot(Y.T, Y) #4*4
    A11 = (A11.T+A11)/2 #4*4
    A13 = np.dot(-Y.T, y).reshape(-1,1) #4*1
    A12 = np.dot(Y.T, z-y).reshape(-1,1) #4*1
    A22 = np.array([np.linalg.norm(z-y)**2]).reshape(1,1) #1*1
    A23 = -1*np.array([np.dot((z-y).T, y)]).reshape(1,1) #1*1
    A33 = np.dot(y.T, y) #1*1
    A1 = np.concatenate([A11, A12, A13], axis = 1)
    A2 = np.concatenate([A12.T, A22, A23], axis = 1)
    A3 = np.concatenate([A13.T, A23, A33], axis = 1)
    A = np.concatenate([A1, A2, A3], axis = 0)
   
    B = np.zeros((n+3, n+3))
    for i in range(n+1, n+3):
        for j in range(n+1, n+3):
            B[i][j]=1
    C = np.zeros((n+3, n+3))
    temp = np.identity(n+1)
    for i in range(n+1):
        for j in range(n+1):
            C[i][j]=1/gamma*temp[i][j]
    for i in range(n+1, n+3):
        for j in range(n+1, n+3):
            C[i][j]=-0.5
    B = B.reshape(1,-1)
    C = -1 * C.reshape(1,-1)
   
    h = [matrix(A.tolist())]
    c = matrix([1.,0.])
    sol = solvers.sdp(c, Gs=[matrix([B.tolist()[0], C.tolist()[0]])], hs=h)
    time2 = time.time()
    return time2-time1
    
    
    
    
    




In [86]:
import random
import numpy as np
def getTime_realData(X, y, z, gamma,size_range):
    fold = 10
    num_size = len(size_range)
    random.seed(2021)
    (m,n) = X.shape
    ssdp_yes = 1
    time_ssdp = np.zeros(num_size)
    time_ssdp_std = np.zeros(num_size)
    optval_ssdp = np.zeros(num_size)
    w_ssdp = np.zeros((n+1,fold))
    for i in range(num_size):
        print('Starting a dataset size of ' + str(i));
        timessdp = np.zeros(fold)
        optvalssdp = np.zeros(fold)
        m_curr = size_range[i]
        for idx in range(fold):
            
            ridx = random.sample(range(m), int(m_curr))
            if ssdp_yes:
                print('Doing ssdp')
                
                timessdp[idx] = sdp(X[ridx] , y[ridx].reshape((-1,1)), z[ridx].reshape((-1,1)), gamma);  
        time_ssdp[i] = np.log(np.mean(timessdp))
        time_ssdp_std[i]= np.std(timessdp)
    return time_ssdp, time_ssdp_std

            
       
           
    
    

In [87]:
import os
import numpy as np
import pandas as pd
from sympy import subsets


# Read data from database provided
# "name" as the name of the dataset
def data_read(name: str, directory="datasets/"):
    if name in ["wine_modest", "wine_severe"]:
        data = pd.read_csv(os.path.join(directory, "wine.csv"),
                           sep=';').to_numpy()
        X, y = data[:, :-1], data[:, -1]
        z = np.clip(y, 0, 10)
        if name.find("modest") != -1:
            z = np.clip(z, 6, None)
        else:
            z = np.clip(z, 7, None)
        # Configuration parameters, can change later using
        # structure options
        const = 0.1
        gamma_list = np.linspace(1e-3, 0.75, 40)
        gamma_time = 0.5
        datasize_list = np.linspace(150, 1500, 16)

    elif name in ["insurance_modest", "insurance_severe"]:
        data = pd.read_csv(os.path.join(directory,
                                        "insurance.csv"), sep=',')
       
        y = data["charges"].to_numpy()
        data = data.drop(columns=['charges'])
        
        for column in ["sex", "smoker", "region"]:
            one_hot = pd.get_dummies(data[column])
            data = data.drop(column, axis=1)
            data = data.join(one_hot)
        X = data.to_numpy()
        if name.find("modest"):
            z = y - 100
        else:
            z = y - 300
        z = np.clip(z, 0, None)
        z /= 100
        y /= 100

        # Configuration parameters
        const = 0.03
        gamma_list = [1e-3]
        gamma_time = 0.5
        datasize_list = np.linspace(100, 1300, 25)

    elif name in ["building_modest", "building_severe",
                  "building_modest170", "building_severe170"]:
        data = pd.read_excel(io=os.path.join(directory, "building.xlsx"),
                             sheet_name="Data", header=1).to_numpy()
        y = data[:, -1]
        X = data[:, :-1]
        if name.find("modest"):
            z = y + 20
        else:
            z = y + 40
        z = np.clip(z, 0, None)

        # Configuration parameters
        const = 0.01
        gamma_list = [20]
        gamma_time = 0.5
        if name in ["building_modest", "building_severe"]:
            datasize_list = np.linspace(100, 300, 21)
        else:
            datasize_list = [170]
        nor = const * np.max(np.abs(y))
        y = y / nor
        z = z / nor

    elif name in ["blog_modest", "blog_severe"]:
        data = pd.read_csv(os.path.join(directory, "blog.csv"),
                           sep=',').to_numpy()
        y = data[:, -1]
        X = data[:, :-1]
        if name.find("modest"):
            z = y + 5
        else:
            z = y + 10
        z = np.clip(z, 0, None)

        # Configuration parameters
        const = 0.01
        gamma_list = [200]
        gamma_time = 0.5
        datasize_list = np.linspace(5000, 50000, 10)

    else:
        raise ValueError("Data set name not recognized.")

    return X, y, z, const, gamma_list, gamma_time, datasize_list

NameError: name 'data_read' is not defined

In [88]:
from matplotlib.pyplot import table
import pandas as pd
from sklearn.preprocessing import normalize
dataname_list = ["wine_modest", "wine_severe", "insurance_modest","insurance_severe"]
len_name = len(dataname_list)
gamma_list = [1e-1]
normalize_yes = 1

for gamma_idx in range(len(gamma_list)):
    gamma = gamma_list[gamma_idx]
    ssdp_mr = np.zeros((len_name,1))
    ssdp_std = np.zeros((len_name,1))
    for idx in range(len_name):
        dataname = dataname_list[idx]
        [X, y, z, const, gamma_list, gamma_time, datasize_list] = data_read(name = dataname)
        
        if normalize_yes:
            X = normalize(X, axis=1, norm='l1')
        
        
        ssdp_mr, ssdp_std = getTime_realData(X, y, z, gamma, datasize_list)
        
        
        print("%%%%%%%%%%%%%%%%%%%%%%%%%%%")
        print(ssdp_mr, ssdp_std)
        df = pd.DataFrame({'datasize': datasize_list, 'ssdp_mr': ssdp_mr, 'ssdp_std': ssdp_std})
         
        table_name_cmp ='result/'+str(dataname)+'_time.csv'
        df.to_csv(table_name_cmp)
        


   

Starting a dataset size of 0
Doing ssdp
     pcost       dcost       gap    pres   dres   k/t
 0:  1.0738e+03 -6.7636e+03  3e+04  1e+00  2e+02  1e+00
 1: -3.2554e+04 -5.3304e+05  2e+08  7e+01  1e+04  8e+04
 2: -1.8939e+03 -8.0660e+02  1e+05  2e+00  3e+02  1e+04
 3: -1.2611e+04 -9.6783e+02  6e+04  4e-01  7e+01  1e+04
 4: -1.5218e+06 -7.4803e+04  1e+07  6e-01  1e+02  1e+06
 5: -1.5248e+08 -6.9572e+06  1e+09  7e-01  1e+02  1e+08
Certificate of dual infeasibility found.
Doing ssdp
     pcost       dcost       gap    pres   dres   k/t
 0:  1.0696e+03 -6.7061e+03  3e+04  1e+00  2e+02  1e+00
 1: -3.2419e+04 -5.3146e+05  2e+08  8e+01  1e+04  8e+04
 2: -1.8947e+03 -1.0591e+03  1e+05  2e+00  3e+02  1e+04
 3: -1.3751e+04 -1.2689e+03  7e+04  4e-01  7e+01  2e+04
 4: -1.6347e+06 -9.1402e+04  1e+07  7e-01  1e+02  2e+06
 5: -1.6379e+08 -8.2967e+06  1e+09  8e-01  1e+02  2e+08
Certificate of dual infeasibility found.
Doing ssdp
     pcost       dcost       gap    pres   dres   k/t
 0:  1.0611e+03 -6.694